In [188]:
import dash
import datetime
import numpy as np
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from dash import html,dcc
from dash.dependencies import Input,Output

pio.templates.defualt="plotly_white"

In [189]:
df_raw=pd.read_csv("E:\\RFM Analysis\\rfm_data.csv")
df_raw.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris


In [190]:
df=df_raw.copy()

In [191]:
df.columns

Index(['CustomerID', 'PurchaseDate', 'TransactionAmount', 'ProductInformation',
       'OrderID', 'Location'],
      dtype='object')

In [192]:
df.shape

(1000, 6)

In [193]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


In [194]:
df.describe()

,CustomerID,TransactionAmount,OrderID
count,1000.000000,1000.00000,1000.000000
mean,5554.789000,513.67781,554071.398000
std,2605.014863,286.09870,264695.448814
min,1011.000000,12.13000,100096.000000
25%,3273.000000,257.12000,313152.000000
50%,5538.000000,523.56500,564671.500000
75%,7821.750000,759.86000,783052.250000
max,9991.000000,999.44000,999695.000000


**Handle Data Type**

In [195]:
df.dtypes

CustomerID              int64
PurchaseDate           object
TransactionAmount     float64
ProductInformation     object
OrderID                 int64
Location               object
dtype: object

In [196]:
df["PurchaseDate"]=pd.to_datetime(df["PurchaseDate"],format="%Y/%m/%d")
df["ProductInformation"]=df["ProductInformation"].astype("string")
df["Location"]=df["Location"].astype("string")               

In [197]:
df.dtypes

CustomerID                     int64
PurchaseDate          datetime64[ns]
TransactionAmount            float64
ProductInformation            string
OrderID                        int64
Location                      string
dtype: object

**Check Null Values**

In [198]:
df.isnull().sum()

CustomerID            0
PurchaseDate          0
TransactionAmount     0
ProductInformation    0
OrderID               0
Location              0
dtype: int64

**Check Duplicates**

In [199]:
df.duplicated().sum()

0

CustomerID

In [200]:
df.CustomerID.duplicated().sum()

54

In [201]:
df.CustomerID.nunique()

946

OrderID

In [202]:
df.OrderID.duplicated().sum()

0

In [203]:
df.OrderID.nunique()

1000

**EDA**

In [204]:
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris


In [205]:
#create new features
df["month_name"]=df["PurchaseDate"].dt.month_name()
df["day"]=df["PurchaseDate"].dt.day
df["day_name"]=df["PurchaseDate"].dt.day_name()

In [206]:
dff=df.groupby("month_name")["OrderID"].count().reset_index(name="num_of_orders")
fig=px.pie(data_frame=dff,
          names="month_name",
          values="num_of_orders",
          hole=0.2,
          color="month_name",
          color_discrete_sequence=px.colors.qualitative.Pastel,
          title="Num of orders in every month",
          width=1400,
          template="presentation",
          labels={"month_name":"Month","num_of_orders":"Num of orders"},
          )
fig.update_traces(textposition="inside",textinfo="percent+value",
                  pull=[0,0,0.1],rotation=90,marker=dict(line=dict(width=1)))
print(dff)
fig.show()

  month_name  num_of_orders
0      April            341
1       June            144
2        May            515


In [207]:
dff=df.groupby("day")["OrderID"].count().reset_index(name="num_of_orders")
fig=px.bar(data_frame=dff,
          x="day",
          y="num_of_orders",
          title="Num of orders throughout the month",
          width=1400,
          text="num_of_orders",
          template="presentation",
          labels={"day":"Day","num_of_orders":"Num of orders"})
print(dff)
fig.show()

    day  num_of_orders
0     1             31
1     2             38
2     3             30
3     4             48
4     5             42
5     6             29
6     7             29
7     8             21
8     9             27
9    10             42
10   11             25
11   12             26
12   13             39
13   14             28
14   15             24
15   16             32
16   17             31
17   18             22
18   19             33
19   20             22
20   21             32
21   22             32
22   23             27
23   24             36
24   25             35
25   26             37
26   27             44
27   28             39
28   29             45
29   30             35
30   31             19


In [208]:
dff=df.groupby("day_name")["OrderID"].count().reset_index(name="num_of_orders")
fig=px.bar(data_frame=dff,
          x="day_name",
          y="num_of_orders",
          color="day_name",
          color_discrete_sequence=px.colors.qualitative.Pastel,
          title="Num of orders throughout the week",
          width=1400,
          text="num_of_orders",
          template="presentation",
          category_orders={"day_name":["Saturday","Sunday","Monday","Tuesday","Wednesday","Thursday","Friday"]},
          labels={"day_name":"Day","num_of_orders":"Num of orders"})
fig.update_layout(showlegend=True)
print(dff)
fig.show()

    day_name  num_of_orders
0     Friday            146
1     Monday            146
2   Saturday            152
3     Sunday            128
4   Thursday            142
5    Tuesday            134
6  Wednesday            152


In [209]:
dff=df.groupby("day",as_index=False).agg({"TransactionAmount":np.mean})
fig=px.line(data_frame=dff,
            x="day",
            y="TransactionAmount",
            title="Avg transactions amount during days",
            color_discrete_sequence=["red"],
            template="presentation",
            labels={"day":"Day","TransactionAmount":"AvgTransactionAmount"},
            )
display(dff)
fig.show()

,day,TransactionAmount
0,1,504.949032
1,2,538.806316
2,3,569.481000
3,4,510.420208
4,5,445.767381
5,6,506.917931
6,7,481.482069
7,8,419.453333
8,9,543.941481
9,10,594.513810


In [210]:
dff=df["ProductInformation"].value_counts().reset_index(name="count").rename({"index":"ProductInformation"},axis=1).sort_values("ProductInformation")

fig=px.bar(data_frame=dff,
          x="ProductInformation",
          y="count",
          color="ProductInformation",
          color_discrete_sequence=px.colors.qualitative.Pastel,
          title="Distribution of ProductInformation",
          width=1400,
          text="count",
          template="presentation",
          labels={"count":"Count"})
fig.update_layout(showlegend=True)
display(dff)
fig.show()

,ProductInformation,count
3,Product A,225
2,Product B,245
0,Product C,277
1,Product D,253


In [211]:
dff=df.groupby("ProductInformation").aggregate({"TransactionAmount":np.mean}).reset_index().rename(columns={"TransactionAmount":"Avg_of_TransactionAmount"})
fig=px.bar(data_frame=dff,
          x="ProductInformation",
          y="Avg_of_TransactionAmount",
          color="ProductInformation",
          color_discrete_sequence=px.colors.qualitative.Pastel,
          title="Avg of TransactionAmount for each ProductInformation",
          width=1400,
          text="Avg_of_TransactionAmount",
          template="presentation",
          labels={"Avg_of_TransactionAmount":"Avg of TransactionAmount"})
fig.update_layout(showlegend=True)
fig.update_traces(texttemplate="%{text:0.4}")
display(dff)
fig.show()

,ProductInformation,Avg_of_TransactionAmount
0,Product A,521.866356
1,Product B,502.196245
2,Product C,521.629928
3,Product D,508.807549


In [212]:
dff=df["Location"].value_counts().reset_index(name="count").rename({"index":"Location"},axis=1).sort_values("Location")

fig=px.bar(data_frame=dff,
          x="Location",
          y="count",
          color="Location",
          color_discrete_sequence=px.colors.qualitative.Pastel,
          title="Distribution of customer locations",
          width=1400,
          text="count",
          template="presentation",
          labels={"count":"Count","Location":"Location of customers"})
fig.update_layout(showlegend=True)
display(dff)
fig.show()

,Location,count
2,London,246
1,New York,247
3,Paris,229
0,Tokyo,278


In [213]:
dff=df.groupby("Location").aggregate({"TransactionAmount":np.mean}).reset_index().rename(columns={"TransactionAmount":"Avg_of_TransactionAmount"})
fig=px.bar(data_frame=dff,
          x="Location",
          y="Avg_of_TransactionAmount",
          color="Location",
          color_discrete_sequence=px.colors.qualitative.Pastel,
          title="Avg of TransactionAmount for each location",
          width=1400,
          text="Avg_of_TransactionAmount",
          template="presentation",
          labels={"Avg_of_TransactionAmount":"Avg of TransactionAmount"})
fig.update_layout(showlegend=True)
fig.update_traces(texttemplate="%{text:0.4}")
display(dff)
fig.show()

,Location,Avg_of_TransactionAmount
0,London,527.633740
1,New York,534.829109
2,Paris,491.008996
3,Tokyo,501.208849


**RFM Analysis**

In [218]:
today=datetime.datetime.today()
today

datetime.datetime(2024, 5, 13, 17, 26, 53, 933849)

In [219]:
rece_data=df.groupby("CustomerID").agg( {"PurchaseDate":lambda PDate: ((today)-(PDate.max())).days} ).reset_index()     # Calculate Recency
rece_data=rece_data.rename(columns={"PurchaseDate":"Recency"})
rece_data

,CustomerID,Recency
0,1011,371
1,1025,359
2,1029,338
3,1046,381
4,1049,351
...,...,...
941,9941,380
942,9950,376
943,9954,350
944,9985,395


In [220]:
freq_data=df.groupby("CustomerID")["OrderID"].count().reset_index(name="Frequency")                                       # Calculate Frequency
freq_data

,CustomerID,Frequency
0,1011,2
1,1025,1
2,1029,1
3,1046,1
4,1049,1
...,...,...
941,9941,1
942,9950,1
943,9954,1
944,9985,1


In [221]:
Mone_data=df.groupby("CustomerID")["TransactionAmount"].sum().reset_index(name="Monetory")                                 # Calculate Monetory
Mone_data

,CustomerID,Monetory
0,1011,1129.02
1,1025,359.29
2,1029,704.99
3,1046,859.82
4,1049,225.72
...,...,...
941,9941,960.53
942,9950,679.11
943,9954,798.01
944,9985,36.10


In [222]:
RFM_df=rece_data.merge(freq_data,on="CustomerID",how="left")
RFM_df=RFM_df.merge(Mone_data,on="CustomerID",how="left")
RFM_df

,CustomerID,Recency,Frequency,Monetory
0,1011,371,2,1129.02
1,1025,359,1,359.29
2,1029,338,1,704.99
3,1046,381,1,859.82
4,1049,351,1,225.72
...,...,...,...,...
941,9941,380,1,960.53
942,9950,376,1,679.11
943,9954,350,1,798.01
944,9985,395,1,36.10


In [223]:
RFM_df["RecencyScore"]  =pd.cut(RFM_df["Recency"],bins=5,labels=[5,4,3,2,1])
RFM_df["FrequencyScore"]=pd.cut(RFM_df["Frequency"],bins=5,labels=[1,2,3,4,5])
RFM_df["MonetoryScore"] =pd.cut(RFM_df["Monetory"],bins=5,labels=[1,2,3,4,5])
RFM_df

,CustomerID,Recency,Frequency,Monetory,RecencyScore,FrequencyScore,MonetoryScore
0,1011,371,2,1129.02,3,3,3
1,1025,359,1,359.29,4,1,1
2,1029,338,1,704.99,5,1,2
3,1046,381,1,859.82,2,1,2
4,1049,351,1,225.72,4,1,1
...,...,...,...,...,...,...,...
941,9941,380,1,960.53,2,1,3
942,9950,376,1,679.11,2,1,2
943,9954,350,1,798.01,5,1,2
944,9985,395,1,36.10,1,1,1


In [224]:
RFM_df.dtypes

CustomerID           int64
Recency              int64
Frequency            int64
Monetory           float64
RecencyScore      category
FrequencyScore    category
MonetoryScore     category
dtype: object

In [225]:
RFM_df["RecencyScore"]=RFM_df["RecencyScore"].astype(int)  
RFM_df["FrequencyScore"]=RFM_df["FrequencyScore"].astype(int)     
RFM_df["MonetoryScore"]=RFM_df["MonetoryScore"].astype(int)     
    
RFM_df.dtypes

CustomerID          int64
Recency             int64
Frequency           int64
Monetory          float64
RecencyScore        int32
FrequencyScore      int32
MonetoryScore       int32
dtype: object

In [226]:
RFM_df["RFMScore"]=RFM_df["RecencyScore"]+RFM_df["FrequencyScore"]+RFM_df["MonetoryScore"]
RFM_df

,CustomerID,Recency,Frequency,Monetory,RecencyScore,FrequencyScore,MonetoryScore,RFMScore
0,1011,371,2,1129.02,3,3,3,9
1,1025,359,1,359.29,4,1,1,6
2,1029,338,1,704.99,5,1,2,8
3,1046,381,1,859.82,2,1,2,5
4,1049,351,1,225.72,4,1,1,6
...,...,...,...,...,...,...,...,...
941,9941,380,1,960.53,2,1,3,6
942,9950,376,1,679.11,2,1,2,5
943,9954,350,1,798.01,5,1,2,8
944,9985,395,1,36.10,1,1,1,3


In [227]:
print("Max",RFM_df["RFMScore"].max())
print("Min",RFM_df["RFMScore"].min())

Max 15
Min 3


In [228]:
RFM_df["Value Segment"]=pd.qcut(RFM_df["RFMScore"],q=3,labels=["Low Value","Mid Value","High Value"])
RFM_df

,CustomerID,Recency,Frequency,Monetory,RecencyScore,FrequencyScore,MonetoryScore,RFMScore,Value Segment
0,1011,371,2,1129.02,3,3,3,9,High Value
1,1025,359,1,359.29,4,1,1,6,Mid Value
2,1029,338,1,704.99,5,1,2,8,High Value
3,1046,381,1,859.82,2,1,2,5,Low Value
4,1049,351,1,225.72,4,1,1,6,Mid Value
...,...,...,...,...,...,...,...,...,...
941,9941,380,1,960.53,2,1,3,6,Mid Value
942,9950,376,1,679.11,2,1,2,5,Low Value
943,9954,350,1,798.01,5,1,2,8,High Value
944,9985,395,1,36.10,1,1,1,3,Low Value


In [229]:
def RFM_Segments(rfm):
    if rfm >= 9 :
        return "Champions"
    elif rfm>=6 and rfm<9 :
        return "Potential Loyalists"
    elif rfm>=5 and rfm<6:
        return "At Risk Customers"
    elif rfm>=4 and rfm<5:
        return "Can't Lose"
    else:
        return "Lost"

RFM_df["RFM Customer Segment"]=RFM_df["RFMScore"].apply(RFM_Segments)
RFM_df

,CustomerID,Recency,Frequency,Monetory,RecencyScore,FrequencyScore,MonetoryScore,RFMScore,Value Segment,RFM Customer Segment
0,1011,371,2,1129.02,3,3,3,9,High Value,Champions
1,1025,359,1,359.29,4,1,1,6,Mid Value,Potential Loyalists
2,1029,338,1,704.99,5,1,2,8,High Value,Potential Loyalists
3,1046,381,1,859.82,2,1,2,5,Low Value,At Risk Customers
4,1049,351,1,225.72,4,1,1,6,Mid Value,Potential Loyalists
...,...,...,...,...,...,...,...,...,...,...
941,9941,380,1,960.53,2,1,3,6,Mid Value,Potential Loyalists
942,9950,376,1,679.11,2,1,2,5,Low Value,At Risk Customers
943,9954,350,1,798.01,5,1,2,8,High Value,Potential Loyalists
944,9985,395,1,36.10,1,1,1,3,Low Value,Lost


In [272]:
segments_count=RFM_df["Value Segment"].value_counts().reset_index()
segments_count.columns=["Value Segment","Count"]

fig_valuesegments=px.pie(
           data_frame=segments_count,
           names="Value Segment",
           values="Count",
           color="Value Segment",
           title="RFM Value Segment Distribution",
           hole=0.3,template="presentation",
           color_discrete_sequence=["green","blue","red"],
           )

fig_valuesegments.update_traces(textposition="outside",textinfo="percent+label+value",pull=[0,0,0.1],marker=dict(line=dict(color="black",width=2)))

display(segments_count)
fig_valuesegments.show()

,Value Segment,Count
0,Low Value,434
1,Mid Value,363
2,High Value,149


In [273]:
dff=RFM_df.groupby("RFM Customer Segment").size().reset_index(name="Count")

fig_rfmsegments=px.bar(data_frame=dff,
           x="RFM Customer Segment",
           y="Count",
           color="RFM Customer Segment",
           title="Comparison of RFM Segments",
           text="Count",
           template="presentation",
           orientation="v",
           labels={"Count":"Num of Customers"},
           category_orders= {"RFM Customer Segment":["Champions","Potential Loyalists","At Risk Customers","Can't Lose","Lost"]}
             )

display(dff)
fig_rfmsegments.show()

,RFM Customer Segment,Count
0,At Risk Customers,179
1,Can't Lose,173
2,Champions,44
3,Lost,82
4,Potential Loyalists,468


In [274]:
dff2=RFM_df.groupby(["Value Segment","RFM Customer Segment"]).size().reset_index(name="Count")

fig_value_rfm_segments=px.treemap(data_frame=dff2,
               path=["Value Segment","RFM Customer Segment"],
               values="Count",
               title="Distribution of RFM Customer Segments within value Segments",
               color="Value Segment",
               color_discrete_sequence=px.colors.qualitative.Pastel
)

display(dff2)
fig_value_rfm_segments.show()

,Value Segment,RFM Customer Segment,Count
0,Low Value,At Risk Customers,179
1,Low Value,Can't Lose,173
2,Low Value,Champions,0
3,Low Value,Lost,82
4,Low Value,Potential Loyalists,0
5,Mid Value,At Risk Customers,0
6,Mid Value,Can't Lose,0
7,Mid Value,Champions,0
8,Mid Value,Lost,0
9,Mid Value,Potential Loyalists,363


In [275]:
dff3=RFM_df.groupby("RFM Customer Segment")[["RecencyScore","FrequencyScore","MonetoryScore"]].mean().reset_index()

fig_rfmsegments_scores=go.Figure()
fig_rfmsegments_scores.add_traces(go.Bar(x=dff3["RFM Customer Segment"],
                      y=dff3["RecencyScore"],
                      marker=dict(color="red"),
                      name="Recency"))
fig_rfmsegments_scores.add_traces(go.Bar(x=dff3["RFM Customer Segment"],
                      y=dff3["FrequencyScore"],
                      marker=dict(color="blue"),
                      name="Frequency"))
fig_rfmsegments_scores.add_traces(go.Bar(x=dff3["RFM Customer Segment"],
                      y=dff3["MonetoryScore"],
                      marker=dict(color="green"),
                      name="Monetory"))
fig_rfmsegments_scores.update_layout(xaxis_title="RFM Segments",yaxis_title="Score",title="Comparison of RFM Segments based on Recency,Frequency and Monetory Scores",showlegend=True)

display(dff3)
fig_rfmsegments_scores.show()

,RFM Customer Segment,RecencyScore,FrequencyScore,MonetoryScore
0,At Risk Customers,2.351955,1.000000,1.648045
1,Can't Lose,1.537572,1.000000,1.462428
2,Champions,4.227273,2.818182,2.977273
3,Lost,1.000000,1.000000,1.000000
4,Potential Loyalists,4.085470,1.059829,1.692308


**Champion Segment**

In [276]:
champion_segment=RFM_df[RFM_df["RFM Customer Segment"]=="Champions"]

In [277]:
fig=go.Figure()
fig.add_trace(go.Box(y=champion_segment.RecencyScore,name="Recency"))
fig.add_trace(go.Box(y=champion_segment.FrequencyScore,name="Frequency"))
fig.add_trace(go.Box(y=champion_segment.MonetoryScore,name="Monetary"))
fig.update_layout(title="Distribution of RFM Values within Champion Segment",
                  yaxis_title="RFM Value",
                  showlegend=True)
fig.show()

In [278]:
correlation_matrix=champion_segment[["RecencyScore","FrequencyScore","MonetoryScore"]].corr()

fig_corrmatrix=px.imshow(img=correlation_matrix.values,
              x=correlation_matrix.columns,
              y=correlation_matrix.columns,
              text_auto=True,
              aspect="auto",
              color_continuous_scale="RdBu",
              #template="presentation",
              title="correlation matrix for RFM values within champions segment")
              
display(correlation_matrix)
fig_corrmatrix.show()

,RecencyScore,FrequencyScore,MonetoryScore
RecencyScore,1.000000,-0.424339,-0.213970
FrequencyScore,-0.424339,1.000000,0.212925
MonetoryScore,-0.213970,0.212925,1.000000


In [283]:
app=dash.Dash(__name__)

app.layout=html.Div([
      
      html.H1("RFM Analysis Dashboard",className="text-center mb-4"),
      html.Div("Analyzis customer segments based on RFM scores",className="text-center mb-4"),
      html.Br(),
      dcc.Dropdown(id="chart_type_dropdown",
                   options=[{"label":"RFM Value Segment Distribution","value":"value segments"},
                            {"label":"Comparison of RFM Segments","value":"RFM segments"},
                            {"label":"Distribution of RFM customer segments within value segments","value":"Value-RFM segments"},
                            {"label":"Comparision of RFM customer segments based on Recency,Frequency and Monetory scores","value":"RFM_segments values"},
                            {"label":"Correlation matrix for RFM values within champions segment","value":"Correlation matrix"}],
                   value="value segments",
                   className="mb-4"),
      dcc.Graph(id="rfm_chart",className="mb-4")
])

In [284]:
@app.callback(
    Output("rfm_chart","figure"),
    [ Input("chart_type_dropdown","value") ]
)
def update_chart(selected_chart):
    if   selected_chart=="value segments":
        return fig_valuesegments
    elif selected_chart=="RFM segments":
        return fig_rfmsegments
    elif selected_chart=="Value-RFM segments":
        return fig_value_rfm_segments
    elif selected_chart=="RFM_segments values":
        return fig_rfmsegments_scores
    elif selected_chart=="Correlation matrix":
        return fig_corrmatrix

    return fig_valuesegments

In [285]:
if __name__=="__main__":
    app.run_server(port=8080)